# Hugging Face TRL을 사용하여 LoRA 어댑터로 LLM 미세 조정하는 방법

이 노트북은 LoRA(Low-Rank Adaptation) 어댑터를 사용하여 대규모 언어 모델을 효율적으로 미세 조정하는 방법을 보여줍니다. LoRA는 다음과 같은 매개변수 효율적인 미세 조정 기법입니다.
- 사전 훈련된 모델 가중치 고정
- 어텐션 레이어에 작은 훈련 가능한 순위 분해 행렬 추가
- 일반적으로 훈련 가능한 매개변수를 약 90% 줄입니다.
- 메모리 효율적이면서 모델 성능 유지

다룰 내용:
1. 개발 환경 및 LoRA 구성 설정
2. 어댑터 훈련을 위한 데이터 세트 생성 및 준비
3. `trl` 및 `SFTTrainer`와 LoRA 어댑터를 사용하여 미세 조정
4. 모델 테스트 및 어댑터 병합(선택 사항)

## 1. 개발 환경 설정

첫 번째 단계는 trl, transformers, datasets를 포함하여 Hugging Face 라이브러리와 Pytorch를 설치하는 것입니다. 아직 trl에 대해 들어본 적이 없다면 걱정하지 마십시오. 이는 transformers 및 datasets 위에 있는 새로운 라이브러리로, 개방형 LLM을 미세 조정, rlhf, 정렬하는 것을 더 쉽게 만듭니다.


In [ ]:
# Google Colab에서 요구 사항 설치
# !pip install transformers datasets trl huggingface_hub

# Hugging Face에 인증

from huggingface_hub import login

login()

# 편의를 위해 허브 토큰을 HF_TOKEN으로 환경 변수에 만들 수 있습니다.

## 2. 데이터 세트 로드

In [13]:
# 샘플 데이터 세트 로드
from datasets import load_dataset

# TODO: path 및 name 매개변수를 사용하여 데이터 세트 및 구성을 정의합니다.
dataset = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")
dataset

DatasetDict({
    train: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 2260
    })
    test: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 119
    })
})

## 3. `trl` 및 LoRA가 포함된 `SFTTrainer`를 사용하여 LLM 미세 조정

`trl`의 [SFTTrainer](https://huggingface.co/docs/trl/sft_trainer)는 [PEFT](https://huggingface.co/docs/peft/en/index) 라이브러리를 통해 LoRA 어댑터와의 통합을 제공합니다. 이 설정의 주요 이점은 다음과 같습니다.

1. **메모리 효율성**:
   - 어댑터 매개변수만 GPU 메모리에 저장됩니다.
   - 기본 모델 가중치는 고정된 상태로 유지되며 더 낮은 정밀도로 로드할 수 있습니다.
   - 소비자 GPU에서 대규모 모델을 미세 조정할 수 있습니다.

2. **훈련 기능**:
   - 최소한의 설정으로 기본 PEFT/LoRA 통합
   - 메모리 효율성을 더욱 향상시키는 QLoRA(양자화된 LoRA) 지원

3. **어댑터 관리**:
   - 체크포인트 중 어댑터 가중치 저장
   - 어댑터를 기본 모델에 다시 병합하는 기능

예제에서는 LoRA를 사용하며, LoRA와 4비트 양자화를 결합하여 성능 저하 없이 메모리 사용량을 더욱 줄입니다. 설정에는 몇 가지 구성 단계만 필요합니다.
1. LoRA 구성 정의(순위, 알파, 드롭아웃)
2. PEFT 구성으로 SFTTrainer 생성
3. 어댑터 가중치 훈련 및 저장


In [ ]:
# 필요한 라이브러리 가져오기
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# 모델 및 토크나이저 로드
model_name = "HuggingFaceTB/SmolLM2-135M"

model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# 채팅 형식 설정
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# 미세 조정을 저장하거나 업로드할 이름 설정
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

`SFTTrainer`는 `peft`와의 기본 통합을 지원하므로 LoRA 등을 사용하여 LLM을 효율적으로 조정하는 것이 매우 쉽습니다. `LoraConfig`를 만들고 트레이너에게 제공하기만 하면 됩니다.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>연습: 미세 조정을 위한 LoRA 매개변수 정의</h2>
    <p>Hugging Face 허브에서 데이터 세트를 가져와 모델을 미세 조정합니다. </p>
    <p><b>난이도</b></p>
    <p>🐢 임의의 미세 조정을 위한 일반적인 매개변수를 사용합니다.</p>
    <p>🐕 매개변수를 조정하고 가중치 및 편향에서 검토합니다.</p>
    <p>🦁 매개변수를 조정하고 추론 결과의 변화를 보여줍니다.</p>
</div>

In [ ]:
from peft import LoraConfig

# TODO: LoRA 매개변수 구성
# r: LoRA 업데이트 행렬의 순위 차원(작을수록 압축률이 높아짐)
rank_dimension = 6
# lora_alpha: LoRA 레이어의 스케일링 팩터(높을수록 적응력이 강해짐)
lora_alpha = 8
# lora_dropout: LoRA 레이어의 드롭아웃 확률(과적합 방지에 도움)
lora_dropout = 0.05

peft_config = LoraConfig(
    r=rank_dimension,  # 순위 차원 - 일반적으로 4-32 사이
    lora_alpha=lora_alpha,  # LoRA 스케일링 팩터 - 일반적으로 순위의 2배
    lora_dropout=lora_dropout,  # LoRA 레이어의 드롭아웃 확률
    bias="none",  # LoRA의 편향 유형. 해당 편향은 훈련 중에 업데이트됩니다.
    target_modules="all-linear",  # LoRA를 적용할 모듈
    task_type="CAUSAL_LM",  # 모델 아키텍처의 작업 유형
)

훈련을 시작하기 전에 사용할 하이퍼파라미터(`TrainingArguments`)를 정의해야 합니다.

In [ ]:
# 훈련 구성
# QLoRA 논문 권장 사항에 기반한 하이퍼파라미터
args = SFTConfig(
    # 출력 설정
    output_dir=finetune_name,  # 모델 체크포인트를 저장할 디렉터리
    # 훈련 기간
    num_train_epochs=1,  # 훈련 에포크 수
    # 배치 크기 설정
    per_device_train_batch_size=2,  # GPU당 배치 크기
    gradient_accumulation_steps=2,  # 더 큰 유효 배치를 위해 기울기 누적
    # 메모리 최적화
    gradient_checkpointing=True,  # 메모리 절약을 위해 계산량 절충
    # 옵티마이저 설정
    optim="adamw_torch_fused",  # 효율성을 위해 융합된 AdamW 사용
    learning_rate=2e-4,  # 학습률(QLoRA 논문)
    max_grad_norm=0.3,  # 기울기 클리핑 임계값
    # 학습률 스케줄
    warmup_ratio=0.03,  # 워밍업 단계 비율
    lr_scheduler_type="constant",  # 워밍업 후 학습률 일정하게 유지
    # 로깅 및 저장
    logging_steps=10,  # N 단계마다 메트릭 로깅
    save_strategy="epoch",  # 에포크마다 체크포인트 저장
    # 정밀도 설정
    bf16=True,  # bfloat16 정밀도 사용
    # 통합 설정
    push_to_hub=False,  # HuggingFace 허브에 푸시하지 않음
    report_to="none",  # 외부 로깅 비활성화
)

이제 모델 훈련을 시작하기 위해 `SFTTrainer`를 만드는 데 필요한 모든 구성 요소를 갖추었습니다.

In [ ]:
max_seq_length = 1512  # 모델 및 데이터 세트 패킹을 위한 최대 시퀀스 길이

# LoRA 구성으로 SFTTrainer 생성
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    peft_config=peft_config,  # LoRA 구성
    max_seq_length=max_seq_length,  # 최대 시퀀스 길이
    tokenizer=tokenizer,
    packing=True,  # 효율성을 위해 입력 패킹 활성화
    dataset_kwargs={
        "add_special_tokens": False,  # 템플릿에서 처리하는 특수 토큰
        "append_concat_token": False,  # 추가 구분 기호 필요 없음
    },
)

`Trainer` 인스턴스에서 `train()` 메서드를 호출하여 모델 훈련을 시작합니다. 이렇게 하면 훈련 루프가 시작되고 3 에포크 동안 모델이 훈련됩니다. PEFT 방법을 사용하므로 전체 모델이 아닌 조정된 모델 가중치만 저장합니다.

In [ ]:
# 훈련 시작, 모델은 허브 및 출력 디렉터리에 자동으로 저장됩니다.
trainer.train()

# 모델 저장
trainer.save_model()

  0%|          | 0/72 [00:00<?, ?it/s]

TrainOutput(global_step=72, training_loss=1.6402628521124523, metrics={'train_runtime': 195.2398, 'train_samples_per_second': 1.485, 'train_steps_per_second': 0.369, 'total_flos': 282267289092096.0, 'train_loss': 1.6402628521124523, 'epoch': 0.993103448275862})

15k 샘플 데이터 세트로 3 에포크 동안 Flash Attention으로 훈련하는 데 `g5.2xlarge`에서 4:14:36이 걸렸습니다. 인스턴스 비용은 `1.21$/h`이므로 총 비용은 약 `5.3$`입니다.



### LoRA 어댑터를 원본 모델에 병합

LoRA를 사용할 때 기본 모델은 고정된 상태로 유지하면서 어댑터 가중치만 훈련합니다. 훈련 중에는 전체 모델 복사본이 아닌 이러한 경량 어댑터 가중치(약 2-10MB)만 저장합니다. 그러나 배포를 위해 다음과 같은 이유로 어댑터를 기본 모델에 다시 병합할 수 있습니다.

1. **간소화된 배포**: 기본 모델 + 어댑터 대신 단일 모델 파일
2. **추론 속도**: 어댑터 계산 오버헤드 없음
3. **프레임워크 호환성**: 서빙 프레임워크와의 호환성 향상


In [ ]:
from peft import AutoPeftModelForCausalLM


# CPU에서 PEFT 모델 로드
model = AutoPeftModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=args.output_dir,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)

# LoRA 및 기본 모델 병합 및 저장
merged_model = model.merge_and_unload()
merged_model.save_pretrained(
    args.output_dir, safe_serialization=True, max_shard_size="2GB"
)

## 3. 모델 테스트 및 추론 실행

훈련이 끝나면 모델을 테스트합니다. 원본 데이터 세트에서 다른 샘플을 로드하고 간단한 루프와 정확도를 메트릭으로 사용하여 해당 샘플에서 모델을 평가합니다.



<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>보너스 연습: LoRA 어댑터 로드</h2>
    <p>예제 노트북에서 배운 내용을 사용하여 훈련된 LoRA 어댑터를 추론용으로 로드합니다.</p> 
</div>

In [30]:
# 다시 메모리 해제
del model
del trainer
torch.cuda.empty_cache()

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

# PEFT 어댑터로 모델 로드
tokenizer = AutoTokenizer.from_pretrained(finetune_name)
model = AutoPeftModelForCausalLM.from_pretrained(
    finetune_name, device_map="auto", torch_dtype=torch.float16
)
pipe = pipeline(
    "text-generation", model=merged_model, tokenizer=tokenizer, device=device
)

일부 프롬프트 샘플을 테스트하고 모델 성능을 확인해 보겠습니다.

In [34]:
prompts = [
    "What is the capital of Germany? Explain why thats the case and if it was different in the past?",
    "Write a Python function to calculate the factorial of a number.",
    "A rectangular garden has a length of 25 feet and a width of 15 feet. If you want to build a fence around the entire garden, how many feet of fencing will you need?",
    "What is the difference between a fruit and a vegetable? Give examples of each.",
]


def test_inference(prompt):
    prompt = pipe.tokenizer.apply_chat_template(
        [{"role": "user", "content": prompt}],
        tokenize=False,
        add_generation_prompt=True,
    )
    outputs = pipe(
        prompt,
    )
    return outputs[0]["generated_text"][len(prompt) :].strip()


for prompt in prompts:
    print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(prompt)}")
    print("-" * 50)